In [11]:
from bertopic import BERTopic
import spacy
import sys
sys.path.append('..')
from src.extract_corpus import get_corpus
corpus = get_corpus('../data')
nlp = spacy.load('fr_core_news_sm')


In [12]:
model = BERTopic(language='french',  verbose=True)
from typing import List
from extract_corpus import Chunk


texts = []

def split_text(corpus:List[Chunk]):
  current_text = ""
  year_text = []
  for chunk in corpus:
    text = chunk.text + "."
    if len(current_text) + len(text) < 512:
        current_text += text
    else:
        year_text.append(current_text.strip())
        current_text = text
  if current_text:
      year_text.append(current_text.strip())
  return year_text

# def remove_stop_words(text:str):
#   return [token.text for token in nlp(text) if token.is_stop is False]

# ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)
# model = BERTopic(ctfidf_model=ctfidf_model, language="french")
#need to keep the stop words 


# for year in corpus:
#   print("year", year)
#   text = [remove_stop_words(ytext) for ytext in split_text(corpus[year])]
#   # text = " ".join([token.text for token in nlp(text) if token.is_stop is False])
#   texts.append(text)
#   # text = "\n".join([chunk.text for chunk in corpus['1814']])
# # doc = nlp("\n".join([chunk.text for chunk in corpus['1814']]))
# # text = " ".join([tok.text for tok in doc if tok.is_stop == False])


In [13]:
def extract_data(topics):
  response = {}
  for topic in topics:
    for item in model.get_topic(topic):
      if item[0] not in response:
        response[item[0]] = []
      response[item[0]].append(item[1])
  
  for term in response:
    response[term] = {
      "count": len(response[term]),
      "max": max(response[term]),
      "min": min(response[term]),
      "average": sum(response[term]) / len(response[term])
    }
  return response


In [17]:
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.vectorizers import ClassTfidfTransformer

response = {}
for year in ["1830"]:
  try:
    print("\n\n===== year", year)
    #split for each year and in each part of the text and remove stop words and join the text
    texts = [" ".join([tok.text for tok in nlp(text) if tok.is_stop is False]) for text in split_text(corpus[year])] 
    topics, probabilities = model.fit_transform(texts)
    response[year] = extract_data(topics)
    print(response[year])
  except:
    pass

# ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)
# topic_model = BERTopic(ctfidf_model=ctfidf_model)

# vectorizer_model = CountVectorizer(stop_words="french")
# model = BERTopic(vectorizer_model=vectorizer_model, language="french")
# clusters, probs = model.fit_transform(response)




===== year 1830


In [18]:
for year in response:
  print(year, list(response[year].keys()))

In [ ]:
# import datamapplot
# from sklearn.datasets import fetch_20newsgroups
# from sentence_transformers import SentenceTransformer
# from bertopic import BERTopic
# from umap import UMAP


# sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
# embeddings = sentence_model.encode(texts, show_progress_bar=False)

# # Train BERTopic
# topic_model = BERTopic().fit(texts, embeddings)

# # # Reduce dimensionality of embeddings, this step is optional
# reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)

# # # Run the visualization with the original embeddings
# topic_model.visualize_document_datamap(texts, embeddings=embeddings)

# # # # Or, if you have reduced the original embeddings already:
# # # topic_model.visualize_document_datamap(texts, reduced_embeddings=reduced_embeddings)
